In [146]:
import re
import fitz 
import spacy
from spacy.matcher import Matcher
from transformers import LayoutLMForTokenClassification, LayoutLMTokenizer
import torch
import os

In [147]:
pdf_path =  r"C:\Users\Jorge\OneDrive\Documents\proyect\document\gaceta_2237.pdf"

try:
    doc = fitz.open(pdf_path)  # Abrir el PDF
    for page_num in range(len(doc)):  # Iterar sobre páginas
        text = doc[page_num].get_text("text")
        print(f"Página {page_num + 1}:\n{text}\n{'-'*80}")
except Exception as e:
    print("Error al procesar el PDF:", e)

Página 1:
P O N E N C I A S
INFORME DE PONENCIA PARA PRIMER DEBATE DEL PROYECTO DE LEY 
ORGÁNICA NÚMERO 041 DE 2024 CÁMARA
por medio del cual se fortalecen las Juntas Administradoras Locales en Colombia y se dictan otras 
disposiciones.
ACUMULADO CON EL PROYECTO DE LEY ORGÁNICA NÚMERO 264 DE 2024 
CÁMARA
por medio del cual se establecen medidas para el fortalecimiento de las Juntas Administradoras Locales en 
el país y se dictan otras disposiciones.
Bogotá D.C., diciembre de 2024. 
INFORME DE PONENCIA PARA PRIMER DEBATE DEL PROYECTO DE LEY 
ORGÁNICA NO. 041 DE 2024 CÁMARA “POR MEDIO DEL CUAL SE 
FORTALECEN LAS JUNTAS ADMINISTRADORAS LOCALES EN COLOMBIA 
Y SE DICTAN OTRAS DISPOSICIONES” ACUMULADO CON EL PROYECTO 
DE LEY ORGÁNICA No. 264 DE 2024 CÁMARA “POR MEDIO DEL CUAL SE 
ESTABLECEN MEDIDAS PARA EL FORTALECIMIENTO DE LAS JUNTAS 
ADMINISTRADORAS LOCALES EN EL PAÍS Y SE DICTAN OTRAS 
DISPOSICIONES” 
 
DOCTORA: 
ANA PAOLA GARCÍA SOTO  
PRESIDENTE. 
COMISIÓN PRIMERA CONSTITUCIONAL. 
CÁMA

In [148]:
labels = [
    "O",  # "O" significa "Outside" (fuera de cualquier entidad)
    "ISSN",
    "FECHA",
    "ENTIDAD",
    "AÑO_ROMANO",
    "NUMERO_PUBLICACION",
    "DIRECTOR_SENADO_NOMBRE",  
    "DIRECTOR_SENADO_CARGO",   
    "DIRECTOR_CAMARA_NOMBRE", 
    "DIRECTOR_CAMARA_CARGO",
    "TIPO_DOCUMENTO",
    "SUBTITULO",
    "SUBSUBTITULO"
]

# Crear un diccionario para mapear etiquetas a índices
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for idx, label in enumerate(labels)}


In [149]:
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    from spacy.cli import download
    download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

In [150]:
# Cargar el tokenizador y el modelo
tokenizer = LayoutLMTokenizer.from_pretrained('microsoft/layoutlm-base-uncased')
model = LayoutLMForTokenClassification.from_pretrained('microsoft/layoutlm-base-uncased', num_labels=len(labels))

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Procesar el Texto con LayoutLM

### Integration with spacy

In [151]:
def postprocess_with_spacy(text, predicted_labels):
    """
    Post-procesa el texto y las etiquetas predichas usando spaCy.
    """
    doc = nlp(text)
    entities = []
    for token, label in zip(doc, predicted_labels):
        if label != "O":  # Ignorar tokens que no pertenecen a ninguna entidad
            entities.append((token.text, label))
    return entities

### PROCESS THE PDF AND TEXT

In [152]:

def leer_pdf(ruta_pdf):
    """Lee un archivo PDF y devuelve su contenido en texto."""
    documento = fitz.open(ruta_pdf)
    texto = ""
    for pagina in documento:
        texto += pagina.get_text()
    return texto

def preprocesar_texto(texto):
    """Limpia el texto eliminando espacios extra y saltos de línea."""
    if not texto:
        return ""
    texto_limpio = re.sub(r"\s+", " ", texto).strip()
    texto_limpio = texto_limpio.replace('\n', ' ')
    return texto_limpio



In [153]:
def extract_with_layoutlm(text):
    """
    Extrae información estructurada del texto usando LayoutLM.
    """
    inputs = tokenizer(text, return_tensors="pt", add_special_tokens=True)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)
    predicted_labels = [id2label[idx] for idx in predictions[0].tolist()]
    return predicted_labels

## LABEL SUBTITLE

In [154]:
def extraer_subtitulo(texto):
    """
    Extrae el subtítulo del documento (texto en negrita).
    """
    patron_subtitulo = r"\*\*(.*?)\*\*"  # Busca texto entre ** **
    coincidencia = re.search(patron_subtitulo, texto)
    if coincidencia:
        return coincidencia.group(1).strip()
    return None

## LABEL SUBSUBTITLE

In [155]:
def extraer_subsubtitulo(texto):
    """
    Extrae el subsubtítulo del documento (texto en cursiva o entre comillas).
    """
    patron_subsubtitulo = r"_(.*?)_|\"(.*?)\""  # Busca texto entre _ _ o " "
    coincidencia = re.search(patron_subsubtitulo, texto)
    if coincidencia:
        # Devuelve el primer grupo no nulo (cursiva o comillas)
        return next(group for group in coincidencia.groups() if group).strip()
    return None

## TYPE DOCUMENT


In [156]:
def separar_documento(text):
    """
    Separa el texto en secciones basadas en patrones de tipos de documentos.
    """
    pattern_tipo_documento = r"(?i)(PONENCIA|ACTA|PROYECTO DE LEY|INFORME|RESOLUCIÓN|CONCEPTO|PROPOSICIÓN|CONSTANCIA|OBJECIONES|CONCEPTOS JURÍDICOS|LEYES SANCIONADAS|PRESENTACIÓN)(.*)"
    documentos = re.findall(pattern_tipo_documento, text)
    documentos = [{"tipo": doc[0].strip(), "texto": doc[1].strip()} for doc in documentos if doc[1].strip()]
    return documentos   

In [157]:
def extraer_info_seccion(seccion):
    """
    Extrae el título, subtítulo y subsubtítulo de una sección de texto.
    """
    info = {
        "tipo": seccion["tipo"],
        "subtitulo": extraer_subtitulo(seccion["texto"]),
        "subsubtitulo": extraer_subsubtitulo(seccion["texto"])
    }
    return info

## FUNCTION PRINCIPALLY

In [158]:
def extraer_info(info_pdf):
    metameta = {}
    
    try:
        # Leer el PDF
        texto_completo = leer_pdf(info_pdf)
        if not texto_completo:
            print(f"Error: El archivo PDF {info_pdf} está vacío o no se pudo leer.")
            return None

        # Preprocesar el texto
        texto_completo = preprocesar_texto(texto_completo)
        if not texto_completo:
            print(f"Error: El texto preprocesado del PDF {info_pdf} está vacío.")
            return None

        # --- Extracción con LayoutLM ---
        predicted_labels = extract_with_layoutlm(texto_completo)
        if not predicted_labels:
            print(f"Error: No se pudieron predecir etiquetas para el PDF {info_pdf}.")
            return None

        # --- Post-procesamiento con spaCy ---
        entities = postprocess_with_spacy(texto_completo, predicted_labels)
        if not entities:
            print(f"Error: No se pudieron extraer entidades del PDF {info_pdf}.")
            return None
        
        # --- Procesar las entidades
        directores = {"senado": {}, "camara": {}}
        for entity_text, entity_label in entities:
            if entity_label == "ISSN":
                metameta["issn"] = entity_text.replace(" ", "")
            elif entity_label == "FECHA":
                metameta["fecha"] = entity_text
            elif entity_label == "ENTIDAD":
                metameta["entidades"] = [entity_text]
            elif entity_label == "DIRECTOR_SENADO_NOMBRE":
                directores["senado"]["nombre"] = entity_text
            elif entity_label == "DIRECTOR_SENADO_CARGO":
                directores["senado"]["cargo"] = entity_text
            elif entity_label == "DIRECTOR_CAMARA_NOMBRE":
                directores["camara"]["nombre"] = entity_text
            elif entity_label == "DIRECTOR_CAMARA_CARGO":
                directores["camara"]["cargo"] = entity_text
            
        # Agregar los directores al diccionario metameta
        metameta["directores"] = directores

        # --- Separar el texto en secciones ---
        secciones = separar_documento(texto_completo)
        if not secciones:
            print(f"Error: No se pudieron separar las secciones del PDF {info_pdf}.")
            return None

        # --- Procesar cada sección ---
        documentos = []
        for seccion in secciones:
            info_seccion = extraer_info_seccion(seccion)
            documentos.append(info_seccion)

        # Agregar los documentos al diccionario metameta
        metameta["documentos"] = documentos

    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo PDF: {info_pdf}")
        return None
    except Exception as e:
        print(f"Error al procesar el PDF {info_pdf}: {e}")
        return None
            
    return metameta

In [159]:

ruta_gaceta = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328_XXV_110_64.pdf"

# Verificar si el archivo existe
if not os.path.exists(ruta_gaceta):
    print(f"Error: El archivo no existe en la ruta: {ruta_gaceta}")
else:
    print("El archivo existe. Procediendo a leer el PDF...")
    texto_prueba = leer_pdf(ruta_gaceta)
    datos_extraidos = extraer_info(texto_prueba)

    if datos_extraidos:
        print(datos_extraidos)

El archivo existe. Procediendo a leer el PDF...
Error al procesar el PDF GACETA DEL CONGRESO  110  
Lunes, 28 de marzo de 2016 
Página 1
C O M E N TA R I O S
(Artículo 36,  Ley 5ª de 1992)
IMPRENTA   NACIONAL   DE   COLOMBIA
www.imprenta.gov.co
SENADO Y CÁMARA
AÑO XXV - Nº 110 
    Bogotá, D. C., lunes, 28 de marzo de 2016 
 EDICIÓN  DE  64  PÁGINAS
DIRECTORES: 
REPÚBLICA   DE   COLOMBIA
RAMA  LEGISLATIVA  DEL  PODER  PÚBLICO
S E N A D O   D E   L A   R E P Ú B L I C A
JORGE HUMBERTO MANTILLA SERRANO
SECRETARIO  GENERAL  DE  LA  CÁMARA
www.camara.gov.co 
GREGORIO ELJACH PACHECO
SECRETARIO  GENERAL  DEL  SENADO
www.secretariasenado.gov.co 
G A C E T A   D E L   C O N G R E S O
I S S N  0 1 2 3  -  9 0 6 6
COMENTARIOS AL PROYECTO DE LEY NÚMERO 97 DE 2015 SENADO
“por la cual se prohíbe la producción, comercialización, exportación, importación  
y distribución de cualquier variedad de asbesto en Colombia”. 
Mg
Mg
RESEÑA
respecto a la diferencia entre el crisotilo  
y los asbestos anﬁboles
